In [49]:
"""     @author:        Guang Yang
        @mktime:        2014/07/17
        @description:   Preprocess raw .csv data into workable form in DataFrame
"""
import slugify
import folium
import geopy
import copy
import time
import pandas as pd
import numpy as np
from pandas import Series, DataFrame # NOQA
from geopy.geocoders import GoogleV3


csv_file = '/Users/guangyang/Work/project_campaign-map/data/final.csv'

raw_spins_df = pd.read_csv(csv_file)


def all_action(df, cols):
    """ finds all possible action taken by the stations
    
    Args:
        df: the dataframe to look
        cols: column names under which to search
    
    Returns:
        action_list: an ndarray of all possible actions
    
    """
    action_list = []
    for i in cols:
        action_list = np.union1d(df[i].unique().tolist(), action_list)
    return action_list


def clean_rotation(spin_df):
    """ Converts rotations (light, medium) to number of spins per week
    Light -> 2 spins
    Medium -> 4 spins
    
    
    Args:
        rot_df: the dataframe with rotations in light/medium
    
    Returns:
        spin_df: converted dataframe with spin numbers instead of rotations
        
    """
    spin_df_clean = spin_df.replace('Light', 2) \
                           .replace('Medium', 4) \
                           .replace('Added', 'added') \
                           .replace('in for review', 'in_for_review') \
                           .replace('Library', 'library') \
                           .replace('n/a', 'null')
    return spin_df_clean


def text_to_zero(cleaned_spin_df):
    """ Converts rotation status texts (added, library) to 0.
    
    Args:
        cleaned_spin_df: dataframe with status texts
    
    Returns:
        zeroed_spin_df: dataframe with 0 for status texts
        
    """
    zeroed_spin_df = cleaned_spin_df.replace('added', '0') \
                            .replace('mailed', '0') \
                            .replace('received', '0') \
                            .replace('reviewing', '0') \
                            .replace('in_for_review', '0') \
                            .replace('library', '0') \
                            .replace('null', '0') \
                            .replace('passed', '0')
    return zeroed_spin_df

def df_unique(df):
    """ Find unique values from the dataframe
    
    Args:
        df: pandas dataframe
    
    Returns:
        np.ndarray of unique strings: ['', '', '']
    """
    
    return Series(df.values.ravel()).unique()


def get_lat_long(city, state):
    """ Makes api call to get lat/long from city and state names
    
    Args:
        city: city name (space is fine?)
        state: state name (space is fine?)
    
    Returns:
        tuple of floats: (lat, long)
    
    """
    time.sleep(1)
    geolocator = GoogleV3()
    #geolocator = GoogleV3(api_key=None,
    #                      domain='maps.googleapis.com',
    #                      scheme='https',
    #                      client_id=None,
    #                      secret_key=None)
    address, (latitude, longitude) = geolocator.geocode("{city}, {state}".format(city=city,
                                                                                 state=state))
    return [latitude, longitude]


def add_lat_long(row):
    """ Add latitude and longitude into the dataframe (insert them after states)
    
    Args:
        row: df row from raw without lat/long
    
    Returns:
        row: df row with lat/long
    """
    lat_long = get_lat_long(row['city'], row['state'])
    full_row = pd.concat([row[:3],
                          Series(data=lat_long, index=['latitude', 'longitude']),
                          row[3:]])
    return full_row


def reformat_for_d3(row):
    """ Reformats a df row into a dataframe
    
    Args:
        row: a row from the processed spins_df
        
    Returns:
        dataframe: a dataframe 
    """
    pass

# alias party
DataFrame.unique_values = df_unique


# Notes
# the status goes from mailed -> received -> in_for_review -> reviewing -> added -> (library)


In [50]:
all_dates = [u'6/9', u'6/16', u'6/23', u'6/30', u'7/7', u'7/14', u'7/21', u'7/28', u'8/4', u'8/11', u'8/18', u'8/25', u'9/1', 
             u'9/8', u'9/15', u'9/22', u'9/29', u'10/6', u'10/13', u'10/20', u'10/27', u'11/3', u'11/10', u'11/17']

# clean up the status during each week
raw_spins_df_clean = copy.copy(raw_spins_df)
raw_spins_df_clean[all_dates] = clean_rotation(raw_spins_df[all_dates])

# add lat/long for each station based on city, state
spins_df = raw_spins_df_clean.apply(func=add_lat_long, axis=1)

# convert status texts to 0's
# raw_spins_df_zeros = copy.copy(raw_spins_df_clean)
# raw_spins_df_zeros[all_dates] = text_to_zero(raw_spins_df_clean[all_dates])
raw_spins_df_zeros = copy.copy(spins_df)
raw_spins_df_zeros[all_dates] = text_to_zero(spins_df[all_dates])

# aggregate data by state
# state_714 = raw_spins_df_zeros[['state', '7/14']].convert_objects(convert_numeric=True)
# temp_state = state_714.groupby(['state']).sum()

# convert to d3 format
# d3_spins_df = 

In [35]:
date804 = raw_spins_df_zeros[['station', 'latitude', 'longitude', '8/4']]
date804.to_json('date804.json')

In [38]:
raw_spins_df_zeros[['station', 'latitude', 'longitude', '6/16']].to_csv('date609.csv')

In [52]:
for date in all_dates:
    raw_spins_df_zeros[['station', 'latitude', 'longitude', date]].rename(columns={date: 'count'})\
    .to_csv('date{}.csv'.format(slugify.slugify(date)))

In [46]:
raw_spins_df_zeros[all_dates].unique_values()

array(['0', '15', '25', '35', '45', '10', '20', '30', '40', '55', '60', 2,
       4, '7', '8', '9', '2', '3', '6', '12', '14', '16', '18', '4', '28',
       'passed', '29', '39', '38', '46', '11', '32', '5'], dtype=object)

CPU times: user 873 ms, sys: 150 ms, total: 1.02 s
Wall time: 38 s


,station,city,state,latitude,longitude,6/9,6/16,6/23,6/30,7/7,...,9/22,9/29,10/6,10/13,10/20,10/27,11/3,11/10,11/17,comments
0,EASY,Tuscarawas,OH,40.394789,-81.407058,received,received,received,received,received,...,35,45,45,45,45,45,45,45,45,NaN
1,Indie 104,Covina,CA,34.090009,-117.890340,added,10,15,20,25,...,60,60,60,60,60,60,60,60,60,NaN
2,KACV,Amarillo,TX,35.221997,-101.831297,received,received,in_for_review,reviewing,added,...,4,4,4,4,4,4,4,4,4,Medium Rotation: 3-5 spins per week
3,KALA,Davenport,IA,41.523644,-90.577637,received,in_for_review,reviewing,added,2,...,4,4,4,4,4,4,4,4,4,Medium Rotation: 3-5 spins per week
4,KALW,Sunnyvale,CA,37.368830,-122.036350,mailed,mailed,received,in_for_review,reviewing,...,2,2,2,2,2,2,2,2,2,NaN
5,KALX,Berkeley,CA,37.871593,-122.272747,received,received,in_for_review,reviewing,reviewing,...,library,library,library,library,library,library,library,library,library,Added to music library; available for play
6,KAOS,Olympia,WA,47.037874,-122.900695,in_for_review,reviewing,added,2,2,...,4,4,4,4,4,4,4,4,4,NaN
7,KBCS,Bellevue,WA,47.610377,-122.200679,mailed,received,received,in_for_review,reviewing,...,2,2,2,2,2,2,2,2,2,NaN
8,KBEACH,Long Beach,CA,33.770050,-118.193740,received,in_for_review,reviewing,added,2,...,4,4,4,4,4,4,4,4,4,Medium Rotation: 3-5 spins per week
9,KBGA,Missoula,MT,46.878718,-113.996586,mailed,received,received,in_for_review,reviewing,...,2,2,2,2,2,2,2,2,2,NaN
